In [1]:
import os
import json
from openai import OpenAI
import instructor
from pydantic import BaseModel, Field
from typing import List
import numpy as np
from utils import get_structured_feedback, remove_duplicates, Evaluation, CriteraFeedback

In [2]:
openai_client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

instructor_client = instructor.from_openai(openai_client)

In [3]:
remove_duplicates(openai_client=openai_client, strings=['Clear and logical organization of the essay', 'Clear and logical organization'])

Similarity Matrix:  [[1.         0.81707956]
 [0.81707956 1.        ]]
Threshold:  0.82
Strings:  Clear and logical organization of the essay Clear and logical organization
Similarity:  0.82



['Clear and logical organization of the essay']

In [4]:
remove_duplicates(openai_client=openai_client, strings=['i am good', 'i am bad'])

Similarity Matrix:  [[1.         0.68911549]
 [0.68911549 1.        ]]
Threshold:  0.69
Strings:  i am good i am bad
Similarity:  0.69



['i am good', 'i am bad']

In [5]:
remove_duplicates(openai_client, ['Relatable and engaging language used to discuss environmental conservation', 'Effectively frames the argument and makes the stance clear and compelling'])

Similarity Matrix:  [[1.         0.34676925]
 [0.34676925 1.        ]]
Threshold:  0.35
Strings:  Relatable and engaging language used to discuss environmental conservation Effectively frames the argument and makes the stance clear and compelling
Similarity:  0.35



['Relatable and engaging language used to discuss environmental conservation',
 'Effectively frames the argument and makes the stance clear and compelling']

In [6]:
# read messages from user1-messages.json
with open("user1-messages.json", "r") as f:
	chat_history = json.load(f)

In [7]:
feedbacks = get_structured_feedback(instructor_client, chat_history)

### Combining evaluations from a chat session to remove duplicates

In [8]:
def aggregate_feedback(feedbacks: List[Evaluation]):
    # Initialize lists for each feedback category
    aggregated = {
        "introduction": {"strengths": [], "weaknesses": [], "suggestions": []},
        "structure": {"strengths": [], "weaknesses": [], "suggestions": []},
        "argumentation": {"strengths": [], "weaknesses": [], "suggestions": []},
        "evidence": {"strengths": [], "weaknesses": [], "suggestions": []},
        "conclusion": {"strengths": [], "weaknesses": [], "suggestions": []}
    }
    
    # Aggregate all feedbacks
    for feedback in feedbacks:
        for key in aggregated:
            critera_feedback = getattr(feedback, key)
            aggregated[key]['strengths'].extend(critera_feedback.strengths)
            aggregated[key]['weaknesses'].extend(critera_feedback.weaknesses)
            aggregated[key]['suggestions'].extend(critera_feedback.suggestions)
    
    # Deduplicate lists
    for key in aggregated:
        for subkey in aggregated[key]:
            aggregated[key][subkey] = remove_duplicates(openai_client, aggregated[key][subkey])

    # Construct a new Evaluation object
    return Evaluation(
        introduction=CriteraFeedback(**aggregated['introduction']),
        structure=CriteraFeedback(**aggregated['structure']),
        argumentation=CriteraFeedback(**aggregated['argumentation']),
        evidence=CriteraFeedback(**aggregated['evidence']),
        conclusion=CriteraFeedback(**aggregated['conclusion'])
    )

In [9]:
aggregated_feedback = aggregate_feedback(feedbacks)

Similarity Matrix:  [[1.         0.4035456  0.67280283 0.99999953 0.44886185]
 [0.4035456  1.         0.60510941 0.40353679 0.45743736]
 [0.67280283 0.60510941 1.         0.67279783 0.46315122]
 [0.99999953 0.40353679 0.67279783 1.         0.44887515]
 [0.44886185 0.45743736 0.46315122 0.44887515 1.        ]]
Threshold:  0.73
Strings:  Effectively framed the argument in the introduction and conclusion Clear and compelling stance on environmental conservation
Similarity:  0.4

Strings:  Effectively framed the argument in the introduction and conclusion Effectively framing the argument and making the stance clear and compelling
Similarity:  0.67

Strings:  Effectively framed the argument in the introduction and conclusion Effectively framed the argument in the introduction and conclusion
Similarity:  1.0

Strings:  Effectively framed the argument in the introduction and conclusion Clear thesis statement
Similarity:  0.45

Strings:  Clear and compelling stance on environmental conservatio

/Users/13point5/Library/Caches/pypoetry/virtualenvs/magicschool-backend-VB4VKBHD-py3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/13point5/Library/Caches/pypoetry/virtualenvs/magicschool-backend-VB4VKBHD-py3.11/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/13point5/Library/Caches/pypoetry/virtualenvs/magicschool-backend-VB4VKBHD-py3.11/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/13point5/Library/Caches/pypoetry/virtualenvs/magicschool-backend-VB4VKBHD-py3.11/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/13point5/

Similarity Matrix:  [[1.         0.30337171 0.80734032]
 [0.30337171 1.         0.30747374]
 [0.80734032 0.30747374 1.        ]]
Threshold:  0.71
Strings:  Consider providing a bit more detail in the counterargument section Add transition sentences between paragraphs for better flow
Similarity:  0.3

Strings:  Consider providing a bit more detail in the counterargument section Consider developing the counterargument section further
Similarity:  0.81

Strings:  Add transition sentences between paragraphs for better flow Consider developing the counterargument section further
Similarity:  0.31

Similarity Matrix:  [[1.         0.52732049 0.76261053 0.8307889 ]
 [0.52732049 1.         0.61871832 0.50470692]
 [0.76261053 0.61871832 1.         0.75984029]
 [0.8307889  0.50470692 0.75984029 1.        ]]
Threshold:  0.79
Strings:  Clear and logical organization Thoughtful structuring from importance to specific examples to counterargument
Similarity:  0.53

Strings:  Clear and logical organiz

In [10]:
for criteria, feedback in aggregated_feedback:
	print(f"{criteria}:")
	print(f"Strengths: {feedback.strengths}")
	print(f"Weaknesses: {feedback.weaknesses}")
	print(f"Suggestions: {feedback.suggestions}")
	print("\n")

introduction:
Strengths: ['Effectively framed the argument in the introduction and conclusion', 'Clear and compelling stance on environmental conservation', 'Effectively framing the argument and making the stance clear and compelling', 'Clear thesis statement']
Weaknesses: ['N/A']
Suggestions: ['Consider providing a bit more detail in the counterargument section', 'Add transition sentences between paragraphs for better flow']


structure:
Strengths: ['Clear and logical organization', 'Thoughtful structuring from importance to specific examples to counterargument', 'Clear and logical structuring of the essay']
Weaknesses: ['N/A', 'Consider developing the counterargument section further']
Suggestions: ['Add more depth to the counterargument section with details or statistics', 'Include transition sentences to improve the flow', 'Develop more details or statistics about economic development in the counterargument section']


argumentation:
Strengths: ['Use of examples like protecting habi

In [11]:
# read messages from user1-messages.json
with open("user2-messages.json", "r") as f:
	chat_history_user2 = json.load(f)

In [12]:
feedbacks2 = get_structured_feedback(instructor_client, chat_history_user2)

In [13]:
aggregated_feedback2 = aggregate_feedback(feedbacks2)

Similarity Matrix:  [[1.         0.2769438  0.61152814 0.43897268]
 [0.2769438  1.         0.69973319 0.5487873 ]
 [0.61152814 0.69973319 1.         0.36955169]
 [0.43897268 0.5487873  0.36955169 1.        ]]
Threshold:  0.64
Strings:  Relatable and engaging use of conversational language Effective examples highlighting the importance of environmental conservation
Similarity:  0.28

Strings:  Relatable and engaging use of conversational language Relatable and engaging language used for the topic of environmental conservation
Similarity:  0.61

Strings:  Relatable and engaging use of conversational language Effective use of examples to illustrate key points
Similarity:  0.44

Strings:  Effective examples highlighting the importance of environmental conservation Relatable and engaging language used for the topic of environmental conservation
Similarity:  0.7

Strings:  Effective examples highlighting the importance of environmental conservation Effective use of examples to illustrate key

In [14]:
for criteria, feedback in aggregated_feedback2:
	print(f"{criteria}:")
	print(f"Strengths: {feedback.strengths}")
	print(f"Weaknesses: {feedback.weaknesses}")
	print(f"Suggestions: {feedback.suggestions}")
	print("\n")

introduction:
Strengths: ['Relatable and engaging use of conversational language', 'Effective examples highlighting the importance of environmental conservation', 'Effective use of examples to illustrate key points']
Weaknesses: []
Suggestions: ['Consider adding more factual information and statistics for a stronger argument', 'Maintain a balance between conversational tone and formal writing standards']


structure:
Strengths: []
Weaknesses: ['Lack of clear organization with introductions, body paragraphs, and conclusions']
Suggestions: ['Structure the essay with clear introductions, body paragraphs, and conclusions for improved flow and readability']


argumentation:
Strengths: []
Weaknesses: []
Suggestions: ['Incorporate research to make the argument more persuasive']


evidence:
Strengths: []
Weaknesses: []
Suggestions: []


conclusion:
Strengths: []
Weaknesses: []
Suggestions: []




In [15]:
all_aggregated_feedbacks = [aggregated_feedback, aggregated_feedback2]

In [16]:
res = aggregate_feedback(all_aggregated_feedbacks)

Similarity Matrix:  [[1.         0.4035456  0.67289682 0.44891005 0.26271297 0.28834109
  0.3991981 ]
 [0.4035456  1.         0.60509932 0.45742256 0.2377964  0.68950444
  0.28528958]
 [0.67289682 0.60509932 1.         0.46318574 0.32167174 0.32332339
  0.43694209]
 [0.44891005 0.45742256 0.46318574 1.         0.16659449 0.24890315
  0.29614365]
 [0.26271297 0.2377964  0.32167174 0.16659449 1.         0.2769438
  0.43897268]
 [0.28834109 0.68950444 0.32332339 0.24890315 0.2769438  1.
  0.5487873 ]
 [0.3991981  0.28528958 0.43694209 0.29614365 0.43897268 0.5487873
  1.        ]]
Threshold:  0.54
Strings:  Effectively framed the argument in the introduction and conclusion Clear and compelling stance on environmental conservation
Similarity:  0.4

Strings:  Effectively framed the argument in the introduction and conclusion Effectively framing the argument and making the stance clear and compelling
Similarity:  0.67

Strings:  Effectively framed the argument in the introduction and conclus

In [17]:
for criteria, feedback in res:
	print(f"{criteria}:")
	print(f"Strengths: {feedback.strengths}")
	print(f"Weaknesses: {feedback.weaknesses}")
	print(f"Suggestions: {feedback.suggestions}")
	print("\n")

introduction:
Strengths: ['Effectively framed the argument in the introduction and conclusion', 'Clear and compelling stance on environmental conservation', 'Clear thesis statement', 'Relatable and engaging use of conversational language']
Weaknesses: ['N/A']
Suggestions: ['Consider providing a bit more detail in the counterargument section', 'Add transition sentences between paragraphs for better flow', 'Maintain a balance between conversational tone and formal writing standards']


structure:
Strengths: ['Clear and logical organization', 'Thoughtful structuring from importance to specific examples to counterargument']
Weaknesses: ['N/A', 'Consider developing the counterargument section further']
Suggestions: ['Add more depth to the counterargument section with details or statistics', 'Include transition sentences to improve the flow', 'Structure the essay with clear introductions, body paragraphs, and conclusions for improved flow and readability']


argumentation:
Strengths: ['Use o